# First Year Project - ITU CPH

## Project 3 - Effectiveness of Skin Cancer Prediction

This notebook contains all of the code developed for project 3, completing tasks similar to data scientists working for a dermatologist to investigate whether some characteristics of skin lesions can be reliably measure with a smartphone app.

The goal is to measure at least 2 of the following characteristics in a set of skin lesion images; asymmetry, border, and color.

Then, we will try to assess how good the measurements are, by predicting the diagnosis of the skin lesions based on these features.

We will focus on the **Melanoma** form of skin cancer.

Group 3:<br>
Crisanna Cornish (ccor@itu.dk)<br>
Danielle Dequin (ddeq@itu.dk)<br>
Gino Franco Fazzi (gifa@itu.dk)<br>
Moneeca Abru Iftikhar Latif (abml@itu.dk)<br>
Carl August Wismer (cwis@itu.dk)

Created: 07-04-2021<br>
Last Modified: 15-04-2021

# Data Sources

Data was provided by the ISIC challenge data sets. <br>
https://challenge.isic-archive.com/data

Codella N, Gutman D, Celebi ME, Helba B, Marchetti MA, Dusza S, Kalloo A, Liopyris K, Mishra N, Kittler H, Halpern A. "Skin Lesion Analysis Toward Melanoma Detection: A Challenge at the 2017 International Symposium on Biomedical Imaging (ISBI), Hosted by the International Skin Imaging Collaboration (ISIC)". arXiv: 1710.05006 [cs.CV]


# Imports

Libraries used in this notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import skimage

from skimage import morphology
from skimage.morphology import opening

from skimage import measure
from skimage import transform
from skimage.transform import rotate
from skimage import filters
from skimage.filters import sobel
from skimage.segmentation import disk_level_set

import math
from math import pi
from math import sqrt

from PIL import Image

# Constants

Constants to access data on the directory structure of this repository.

In [ ]:
IMG_PATH = './data/example_image/'
SEG_PATH = './data/example_segmentation/'
FEAT_PATH = './features/'

TRUTH = './data/ISIC-2017_Training_Part3_GroundTruth.csv'
FEATURES = '/features/features.csv'

# Functions

Functions created for this notebook.

In [ ]:
"""FUNCTIONS FOR MASKED IMAGES"""

def get_boundaries(image):
    """Function to locate the boundaries of the lesion over the whole image.
    Takes a segmentation mask image as argument and returns the upper, lower, left and right boundaries."""

    mask = np.where(image == 1)
    left = min(mask[1])
    right = max(mask[1])
    upper = min(mask[0])
    lower = max(mask[0])
    return upper, lower, left, right

def get_center(image): # NOT NEEDED ANYMORE ?
    """Function that takes an image as input, and returns the centerpoint of the lesion."""
    up, dw, lt, rt = get_boundaries(image)
    center = ((up+dw)/2, (lt+rt)/2)
    return center
    
def zoom(image):
    """Function to zoom-in (crop) the lesion from blank space. Takes a segmentation mask image as input,
    and returns the rectangle where the lesion is found."""

    up, dw, lt, rt = get_boundaries(image)
    rectangle = image[up:dw+1, lt:rt+1]
    return rectangle

def cuts(image):
    """Function to perform a double cut (vertical and horizontal) of the lesion. Takes a segmentation mask image as input,
    and returns the vertical and horizontal cuts (2 for each dimension). It handles uneven shapes."""

    center_h = image.shape[0] // 2 # The image shape contains a tuple with height and width (in pixels)
    if image.shape[0] % 2 == 0: # If the height is an even number of pixels, the cut returns 2 equal sides
        upside = image[:center_h,:]
        downside = image[center_h:,:]
    else: # If the height is an uneven number of pixels, the cut has to "share" the center, to return 2 equal sides
        upside = image[:center_h,:]
        downside = image[center_h+1:,:]
        
    center_w = image.shape[1] // 2    
    if image.shape[1] % 2 == 0:
        leftside = image[:,:center_w]
        rightside = image[:,center_w:]
    else:
        leftside = image[:,:center_w]
        rightside = image[:,center_w+1:]
 
    return upside, downside, leftside, rightside


def test_symmetry(image, rot_deg=30):
    """Function to test the symmetry of an image. Takes a segmentation mask image and the rotation degree interval and
    returns a symmetry score between zero (non-symmetric) to one (completely symmetric)."""

    assert (rot_deg <= 90) and (rot_deg >= 0), "Rotation degree should be positive and at most 90 deg"
    optimal = 0
    
    for deg in range(0,91, rot_deg):
        rot_image = skimage.transform.rotate(image, deg)
        z = zoom(rot_image)
        
        upside, downside, leftside, rightside = cuts(z)

        up_dw = np.sum(np.bitwise_and(upside.astype(int), np.flipud(downside).astype(int))) /\
        np.sum(np.bitwise_or(upside.astype(int), np.flipud(downside).astype(int)))

        lt_rt = np.sum(np.bitwise_and(leftside.astype(int), np.fliplr(rightside).astype(int))) /\
        np.sum(np.bitwise_or(leftside.astype(int), np.fliplr(rightside).astype(int)))
    
        symmetry = (up_dw+lt_rt)/2
        
        if symmetry > optimal: optimal = symmetry

    return symmetry
    
def rgb2gray(rgb):
    """Function to convert a RGB image to grayscale."""
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def crop(image, mask):
    """"""
    img = image.copy()
    img[mask==0] = 0
    return img

def color_std(image):
    """A function that takes an image as input, computes and returns the average standard deviation of all the
    rgb color values."""
    try:
        R = image[np.where(image[:,:,0] != 0) and np.where(image[:,:,1] != 0) and np.where(image[:,:,2] != 0)][:,0]
        G = image[np.where(image[:,:,0] != 0) and np.where(image[:,:,1] != 0) and np.where(image[:,:,2] != 0)][:,1]
        B = image[np.where(image[:,:,0] != 0) and np.where(image[:,:,1] != 0) and np.where(image[:,:,2] != 0)][:,2]
        color_std = (np.std(R) + np.std(G) + np.std(B)) /3
    except:
        color_std = 'NA'
    return color_std

def check_border(image, border=0.01, tolerance=0.2, warning=True):
    """Function to check if the lesion might be exceeding the image. Take the following arguments:
    - image: segmentation mask image to check.
    - border: the percentage of pixels to consider as a border. 10% by default.
    - tolerance: the percentage of tolerance for a lesion to be at the border of the image. 20% by default.
    - warning: boolean to indicate if a textual warning should be issue when checking the border. True by default."""
    h = int(image.shape[0] * border)
    w = int(image.shape[1] * border)
    up = (np.sum(image[h,:]) / image.shape[1]) > tolerance
    dw = (np.sum(image[-h,:]) / image.shape[1]) > tolerance
    lt = (np.sum(image[:,w]) / image.shape[0]) > tolerance
    rt = (np.sum(image[:,w]) / image.shape[0]) > tolerance
    if warning:
        if up or dw or lt or rt: return "This lesion might be overflowing the image"
        else: return "This lesion does not seem to be overflowing the image"
    else:
        return up or dw or lt or rt
    
"""
#def laydown(image): # I THINK WE MAY NOT NEED THIS
#    z = zoom(image)
#    u, d, l, r = get_boundaries(z)
#    if (d-u) >= (r-l):
#        return skimage.transform.rotate(image, 90) 
#    else: return image
        
def reverse(image):
    new = image.copy()
    new[np.where(image == 1)], new[np.where(image == 0)] = 0, 1
    return new
"""
def masker(image, sens):
    '''Takes image, converts to a grayscale image, and returns a masked 
    image that only shows values below the sensitivity given as input.'''
    
    gray = rgb2gray(image) # Create grayscale image
    img2 = gray < sens # **This level needs manually adjusting, also need to be able to automate**
    
    # use plt.imshow(masker(image,sens), cmap='gray') to see image
    
    return img2.astype(int)

def dimensions(mask1):
    '''calculates height(max) and width(90 deg to height)
        returns height, width, rotated mask image, degree of rotation'''
    pixels_in_col = np.max(np.sum(mask1, axis=0))

    rot = 0
    max_col = 0
    rot_max = 0
    for _ in range(9):
        rot_im = transform.rotate(mask1,rot)
        pixels_in_col = np.max(np.sum(rot_im, axis=0))
        if pixels_in_col > max_col:
            max_col = pixels_in_col
            rot_max = rot
            pixels_in_row = np.max(np.sum(rot_im, axis=1))
        rot += 10

    return max_col, pixels_in_row, rot_max

def measure_area_perimeter(mask, option=1):
    """A function that takes either a segmented image or perimeter 
    image as input, and calculates the length of the perimeter of a lesion."""
    
    # Measure area: the sum of all white pixels in the mask image
    area = np.sum(mask)

    # Measure perimeter: first find which pixels belong to the perimeter.
    if option == 1:
        struct_el = morphology.disk(1)
        mask_eroded = morphology.binary_erosion(mask, struct_el)
        image_perimeter = mask - mask_eroded

        # Now we have the perimeter image, the sum of all white pixels in it
        perimeter = np.sum(image_perimeter)
    else:
        perimeter = measure.perimeter(mask)

    return area, perimeter

def predict(bi_image):
    """"""
    area = np.sum(bi_image)
    _, peri = perimeter(bi_image)
    
    area_from_peri = pi*((peri/(2*pi))**2)
    peri_from_area = 2*pi*sqrt(area/pi)
    
    return area, area_from_peri, peri, peri_from_area

## Exploratory Data Analysis

Here we visualize a couple of images and masks to get familiar with the images and its attributes.

## Dataset Creation

(NOTE) We load and access the metadata included in our source data.

In [ ]:
df = pd.read_csv(TRUTH, index_col='image_id')
df = df.astype(int) # Transform to int
df.head()

In [1]:
df['non-cancer'] = 1 - df.sum(axis=1) # Define a 'non-cancer' label

NameError: name 'df' is not defined

## Feature Extraction

(NOTE) We proceed to extract features of interest for our predictive model

#### Asymmetry

To test for asymmetry we run a function to calculate a score based on how similar an image is when cut horizontally and vertically. We assign a score between 0 (non asymmetric) and 1 (totally asymmetric) for both cuts, and we take the average to convey a unique value.

In [ ]:
WARN = input("This operation may take several minutes. Do you wish to continue: (Yes/No) ")

REWRITE = input("Do you wish to overwrite the /features/symmetry.csv file?: (Yes/No) ")
print("\n----- PLEASE BE PATIENT -----\n")

if WARN.lower().startswith("y"):
    symmetry = {}
    i = 1
    for ix, row in df[:10].iterrows():
        file_path = SEG_PATH + str(ix) + "_segmentation.png"
        image = plt.imread(file_path)
        
        ptg = round((i / 10) * 100,2)
        print(f'\rCalculating symmetry: {ptg}%', end='\r')
        symmetry[ix] = test_symmetry(image)
        i += 1
else: print("OPERATION CANCELLED")
    
if REWRITE.lower().startswith("y"):
    with open(FEAT_PATH + 'symmetry.csv', 'w') as outfile:
        outfile.write('image_id'+','+'symmetry'+'\n')
        for k, v in symmetry.items():
            line = k +','+str(v)
            outfile.write(line+'\n')

#### Border

In [2]:
WARN = input("This operation may take several minutes. Do you wish to continue: (Yes/No) ")

REWRITE = input("Do you wish to overwrite the /features/compactness.csv file?: (Yes/No) ")
print("\n----- PLEASE BE PATIENT -----\n")

if WARN.lower().startswith("y"):
    compactness = {}
    i = 1
    for ix, row in df.iterrows():
        file_path = SEG_PATH + str(ix) + "_segmentation.png"
        image = plt.imread(file_path)
        
        ptg = round(i / len(df.index)*100,2)
        print(f'\rCalculating compactness: {ptg}%', end='\r')
        area, per = measure_area_perimeter(image, option=2)
        compactness[ix] = (4* math.pi * area) / (per**2)
        i += 1
else: print("OPERATION CANCELLED")
    
if REWRITE.lower().startswith("y"):
    with open(FEAT_PATH + 'compactness.csv', 'w') as outfile:
        outfile.write('image_id'+','+'compactness'+'\n')
        for k, v in compactness.items():
            line = k +','+str(v)
            outfile.write(line+'\n')

This operation may take several minutes. Do you wish to continue: (Yes/No) n
Do you wish to overwrite the /features/compactness.csv file?: (Yes/No) n

----- PLEASE BE PATIENT -----

OPERATION CANCELLED


In [ ]:
plt.imshow(test)

In [ ]:
rect = np.ones([4,4])
print(measure_area_perimeter(test, option=2))
lt = np.where(rect == 1)
#print(list(lt[0])+list(lt[1]))
x= len(set(list(lt[0]) + list(lt[1])))
x

In [ ]:

len(set(lt)) + len(set(lt2))

#### Color

### Model

#### Creation

#### Data splitting

#### Feature scalling

#### Model fitting

#### Model prediction

#### Model score